In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os
from PyPDF2 import PdfFileReader, PdfFileWriter
from PIL import Image

def convert_images_to_pdf_with_layout(image_folder, output_pdf, images_per_page=6):
    page_width, page_height = letter
    margin = 50
    image_width = (page_width - margin * 3) / 2  

    c = canvas.Canvas(output_pdf, pagesize=letter)
    image_files = [f for f in sorted(os.listdir(image_folder)) if f.endswith('.png')]
    total_images = len(image_files)
    image_count = 0

    for image_file in image_files:
        column = image_count % 2
        row = (image_count // 2) % 3

        image_path = os.path.join(image_folder, image_file)
        img = Image.open(image_path)
        img_height = (image_width / img.width) * img.height

        x_position = margin + (margin + image_width) * column
        y_position = page_height - margin - (img_height + margin) * row - img_height

        c.drawImage(image_path, x_position, y_position, width=image_width, height=img_height, preserveAspectRatio=True)
        
        image_count += 1
        if image_count % images_per_page == 0 and image_count < total_images:
            c.showPage()

    c.save() 

def append_images_pdf_to_pdf(main_pdf_path, images_pdf_path, final_pdf_path):
    output_pdf = PdfFileWriter()
    with open(main_pdf_path, 'rb') as f:
        main_pdf = PdfFileReader(f)
        images_pdf = PdfFileReader(open(images_pdf_path, 'rb'))

        for i in range(main_pdf.getNumPages()):
            output_pdf.addPage(main_pdf.getPage(i))
        
        for i in range(images_pdf.getNumPages()):
            output_pdf.addPage(images_pdf.getPage(i))
        
        with open(final_pdf_path, 'wb') as f_out:
            output_pdf.write(f_out)

image_folder_path = ''
images_pdf_path = ''
main_pdf_path = ''  
final_pdf_path = ''


convert_images_to_pdf_with_layout(image_folder_path, images_pdf_path)
append_images_pdf_to_pdf(main_pdf_path, images_pdf_path, final_pdf_path)


os.remove(images_pdf_path)
